In [266]:
import polars as pl
import polars.selectors as cs
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from great_tables import style, loc

In [267]:
dataset = pl.read_csv('Data.csv')

In [268]:
dataset

Country,Age,Salary,Purchased
str,i64,i64,str
"""France""",44,72000,"""No"""
"""Spain""",27,48000,"""Yes"""
"""Germany""",30,54000,"""No"""
"""Spain""",38,61000,"""No"""
"""Germany""",40,null,"""Yes"""
"""France""",35,58000,"""Yes"""
"""Spain""",null,52000,"""No"""
"""France""",48,79000,"""Yes"""
"""Germany""",50,83000,"""No"""


In [269]:
# age_average = dataset.select(
#     pl.col('Age').mean().round(0).cast(pl.UInt8)
# )
# age_average

In [270]:
# salary_average = dataset.select(
#     pl.col('Salary').mean().round(0).cast(pl.UInt16)
# )
# salary_average

In [271]:
# dataset = dataset.with_columns(
#     pl.col('Age').fill_null(age_average),
#     pl.col('Salary').fill_null(salary_average),
# )

In [272]:
X = dataset.select(cs.exclude('Purchased'))

In [273]:
y = dataset.select('Purchased')

In [274]:
from sklearn.impute import SimpleImputer

In [275]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [276]:
numeric_data = X.select(cs.numeric())
non_numeric_data = X.select(cs.exclude(cs.numeric()))

In [277]:
imputer.fit(numeric_data)

SimpleImputer()

In [278]:
imputed_data = imputer.transform(numeric_data)

In [279]:
imputed_data = pl.DataFrame(imputed_data, schema=numeric_data.schema)

In [280]:
X = pl.concat([non_numeric_data, imputed_data], how='horizontal')

In [281]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [282]:
ct = ColumnTransformer(
    transformers=[('encoder', OneHotEncoder(), [0])],
    remainder='passthrough',
)

In [283]:
X = np.array(ct.fit_transform(X.to_pandas()))

In [284]:
X

array([[1.0000e+00, 0.0000e+00, 0.0000e+00, 4.4000e+01, 7.2000e+04],
       [0.0000e+00, 0.0000e+00, 1.0000e+00, 2.7000e+01, 4.8000e+04],
       [0.0000e+00, 1.0000e+00, 0.0000e+00, 3.0000e+01, 5.4000e+04],
       [0.0000e+00, 0.0000e+00, 1.0000e+00, 3.8000e+01, 6.1000e+04],
       [0.0000e+00, 1.0000e+00, 0.0000e+00, 4.0000e+01, 6.3777e+04],
       [1.0000e+00, 0.0000e+00, 0.0000e+00, 3.5000e+01, 5.8000e+04],
       [0.0000e+00, 0.0000e+00, 1.0000e+00, 3.8000e+01, 5.2000e+04],
       [1.0000e+00, 0.0000e+00, 0.0000e+00, 4.8000e+01, 7.9000e+04],
       [0.0000e+00, 1.0000e+00, 0.0000e+00, 5.0000e+01, 8.3000e+04],
       [1.0000e+00, 0.0000e+00, 0.0000e+00, 3.7000e+01, 6.7000e+04]])

In [285]:
from sklearn.preprocessing import LabelEncoder

In [286]:
le = LabelEncoder()

In [287]:
y = y.to_numpy().ravel()

In [288]:
y = le.fit_transform(y)

In [289]:
from sklearn.model_selection import train_test_split

In [290]:
X_train, X_test, y_train , y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [291]:
from sklearn.preprocessing import StandardScaler

In [292]:
scaler = StandardScaler()

In [293]:
scaled_X_train = scaler.fit_transform(X_train)

In [294]:
scaled_X_test = scaler.transform(X_test)

In [295]:
scaled_X_train

array([[-0.77459667, -0.57735027,  1.29099445, -0.28942984, -1.07811699],
       [-0.77459667,  1.73205081, -0.77459667,  0.        , -0.07018988],
       [ 1.29099445, -0.57735027, -0.77459667,  0.57885968,  0.63357038],
       [-0.77459667, -0.57735027,  1.29099445, -0.28942984, -0.30785767],
       [-0.77459667, -0.57735027,  1.29099445, -1.88129397, -1.42045447],
       [ 1.29099445, -0.57735027, -0.77459667,  1.15771937,  1.23266096],
       [-0.77459667,  1.73205081, -0.77459667,  1.44714921,  1.57499844],
       [ 1.29099445, -0.57735027, -0.77459667, -0.72357461, -0.56461078]])

In [296]:
scaled_X_test

array([[-0.77459667,  1.73205081, -0.77459667, -1.44714921, -0.90694825],
       [ 1.29099445, -0.57735027, -0.77459667, -0.43414476,  0.20564854]])